In [ ]:
import pandas as pd
import ast

input_file = 'hebrew_dict/hebrew_verbs.csv'

df = pd.read_csv(input_file)

# filter by roots, binyan, and part of speech
binyans = ["pa'al", "nif'al", "pi'el", "hif'il", "hitpa'el"]
binyan = binyans[0]

df['Root'] = df['Root'].apply(ast.literal_eval)
# df = df[df['Root'].apply(lambda x: len(x) == 3)]
df = df[df['Binyan'].apply(lambda x: x == binyan)]
df = df[df['Part of speech'].apply(lambda x: x == "Verb")]

In [ ]:
def replace_final_form(hebrew_chars):
    # Define mapping of normal form characters to their final forms
    # reversed due to hebrew direction
    final_form_mapping = {
        "ץ": "צ",
        "ף": "פ",
        "ן": "נ",
        "ם": "מ",
        "ך": "כ"
    }

    # Replace final forms in the list
    return ''.join([final_form_mapping.get(char, char) for char in hebrew_chars])

In [ ]:
import networkx as nx
from collections import Counter

G = nx.Graph()

node_counter_dict = {}

for index, row in df.iterrows():
    root = ''.join(row['Root'])
    word = row['Word (he)']
    meaning = row['Meaning']

    G.add_node(root, title=meaning, word=word)
    node_counter_dict[root] = Counter(replace_final_form(root))

In [ ]:
import itertools
from Levenshtein import distance


G.remove_edges_from(list(G.edges))


def edge_levenshtein_k(n, m, k):
    return distance(replace_final_form(n), replace_final_form(m)) <= k


def is_permutation(n, m):
    return node_counter_dict[n] == node_counter_dict[m]


for n, m in itertools.combinations(G.nodes(), 2):
    if is_permutation(n, m):  # edge_levenshtein_k(n, m, 1):
        G.add_edge(n, m)

In [ ]:
output_fname = f"roots_verb_{binyan}"

nx.write_gexf(G, f"graphs/{output_fname}.gexf")

In [ ]:
from pyvis.network import Network

# Create a new pyvis network
net = Network(notebook=True, bgcolor="#222222",
              font_color="white", height="100vh")

# Load the graph from the GEXF file
# G = nx.read_gexf("roots_verb_paal_3char.gexf")

# filter degree

nonzero_degree_nodes = [node for node, degree in G.degree if degree != 0]

# Create a new graph with only nodes of nonzero degree
G = G.subgraph(nonzero_degree_nodes)

net.from_nx(G)

# Set some options for the network
net.options = {
    "nodes": {
        "font": {
            "size": 36,
            "face": "tahoma"
        },
        "shape": "dot",
        "size": 10,
        "color": {
            "border": "#2B7CE9",
            "background": "#666",
            "highlight": {
                "border": "#2B7CE9",
                "background": "#848484"
            }
        },
        "shadow": {
            "enabled": True,
            "size": 15
        },
        "title": "meaning"  # Hover attribute to display semantic meaning
    },
    "edges": {
        "color": {
            "color": "#ccc",
            "highlight": "#848484"
        }
    },
    "interaction": {
        "hover": True,
        "navigationButtons": True,
        "zoomView": True,
        "dragView": True
    },
    "physics": {
        "enabled": True
    }
}

# Show the network
net.show(f"html/{output_fname}_network.html")